### File data.general avec 'info_student' & 'Sondage_LV2'

In [40]:

import sqlite3
import pandas as pd
import numpy as np
import json
import os


depot_info_folder = './data/depot_info'
db_path = './data/SQL/data.sqlite3' 


def file_data_general(depot_info_folder, db_path):
    conn = sqlite3.connect(db_path)
    desired_table_name = 'General'
    for file in [f for f in os.listdir(depot_info_folder)]:
        if os.path.splitext(os.path.basename(file))[0] == 'Info_student' :
            db_column_mapping = {
                'Nom': 'NAME',
                'Prénom': 'SURNAME',
                'mail': 'EMAIL',
                'Class': 'CLASS'}

            if file.endswith('.csv'):
                csv_file_path = os.path.join(depot_info_folder, file)
                df = pd.read_csv(csv_file_path, encoding='utf-8-sig')                
                df.rename(columns=db_column_mapping, inplace=True)
                df = df[list(db_column_mapping.values())]
                df.to_sql(desired_table_name, conn, if_exists='append', index=False)

            elif file.endswith('.json'):
                json_file_path = os.path.join(depot_info_folder, file)
                with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
                    data = json.load(json_file)
                    transformed_data = []
                    for item in data:
                        transformed_item = {db_column: item.get(json_field) for json_field, db_column in db_column_mapping.items()}
                        transformed_data.append(transformed_item)
                    df = pd.DataFrame(transformed_data)
                    df.to_sql(desired_table_name, conn, if_exists='append', index=False)

            elif file.endswith('.xlsx'):
                xlsx_file_path = os.path.join(depot_info_folder, file)
                df = pd.read_excel(xlsx_file_path)
                df.rename(columns=db_column_mapping, inplace=True)
                df = df[list(db_column_mapping.values())]
                df.to_sql(desired_table_name, conn, if_exists='append', index=False)

            else:
                print(f"Format de fichier non pris en charge: {file}")

        elif os.path.splitext(os.path.basename(file))[0] == 'Sondage_LV2' :
            db_column_mapping = {
                'Nom': 'NAME',
                'Prénom': 'SURNAME',
                'mail': 'EMAIL',
                'Langues' : 'LV2'
            }
            if file.endswith('.csv'):
                csv_file_path = os.path.join(depot_info_folder, file)
                with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                    csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                    for ligne in csv_reader:
                        ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['Nom']} {ligne_filtered['Prénom']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET LV2=?, STATUS=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["Langues"],"PRESENT", ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        conn.commit()

            elif file.endswith('.json'):
                json_file_path = os.path.join(depot_info_folder, file)
                with open(json_file_path, 'r', encoding='utf-8-sig') as json_file:
                    data = json.load(json_file)
                    for ligne in data:
                        ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['Nom']} {ligne_filtered['Prénom']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET LV2=?, STATUS=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["Langues"],"PRESENT", ligne_filtered["Nom"], ligne_filtered["Prénom"], ligne_filtered["mail"]))
                        conn.commit()

            elif file.endswith('.xlsx'):
                xlsx_file_path = os.path.join(depot_info_folder, file)
                df = pd.read_excel(xlsx_file_path)
                df = df[list(db_column_mapping.keys())]
                cursor_destination = conn.cursor()
                for index, ligne in df.iterrows():
                    cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne["Nom"], ligne["Prénom"], ligne["mail"]))
                    count = cursor_destination.fetchone()[0]
                    if count == 0:
                        print(f"{ligne['Nom']} {ligne['Prénom']} doesn't exist")
                    else:
                        cursor_destination.execute("UPDATE General SET LV2=?, STATUS=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["Langues"],"PRESENT", ligne["Nom"], ligne["Prénom"], ligne["mail"]))
                conn.commit()

            else:
                print(f"Format de fichier non pris en charge: {file}")

    conn.close()
    return

file_data_general(depot_info_folder,db_path)

### File data.general avec $note_langue

comment savoir TT pour etudiant debutant dans un langue avec sondage anglais ???

In [41]:
import sqlite3
import pandas as pd
import numpy as np
import json
import os

def load_survey(depot_note_folder,db_path ):
    conn = sqlite3.connect(db_path)

    for file in [f for f in os.listdir(depot_note_folder)]:

        if "Anglais" not in os.path.splitext(os.path.basename(file))[0] :
            db_column_mapping = {
                    'Nom': 'NAME',
                    'Prénom': 'SURNAME',
                    'Mail': 'EMAIL',
                    'Note/10' : 'GRADE_LV2'
                    }
            
            if file.endswith('.csv'):
                csv_file_path = os.path.join(depot_note_folder, file)
                with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                    csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                    csv_reader['Note/10'] = csv_reader['Note/10'].replace('', np.nan)
                    csv_reader['Note/10'] = csv_reader['Note/10'].astype(float)
                    for ligne in csv_reader:
                        ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))                    
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET GRADE_LV2=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV2"], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()
                            
            elif file.endswith('.json'):
                json_file_path = os.path.join(depot_note_folder, file)
                with open(json_file_path, 'r', encoding='utf-8-sig') as jsonfile:
                    data = json.load(jsonfile)
                    df = pd.DataFrame(data)
                    df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                    for index, row in df.iterrows():
                        ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;",(ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                            cursor_destination.execute("UPDATE General SET GRADE_LV2=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV2"], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()

            elif file.endswith('.xlsx'):
                xlsx_file_path = os.path.join(depot_note_folder, file)
                df = pd.read_excel(xlsx_file_path)
                df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                for index, row in df.iterrows():
                    ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                    cursor_destination = conn.cursor()
                    cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                    count = cursor_destination.fetchone()[0]
                    if count == 0:
                        print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                    else:
                        cursor_destination.execute("UPDATE General SET GRADE_LV2=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV2"], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                    conn.commit()
                    
            else:
                print(f"Format de fichier non pris en charge: {file}")
        
        else :
            db_column_mapping = {
                'Nom': 'NAME',
                'Prénom': 'SURNAME',
                'Mail': 'EMAIL',
                'Note/10' : 'GRADE_LV1'
            }
            if "_TT" in os.path.splitext(os.path.basename(file))[0]: 
                
                extra_time = True
                if file.endswith('.csv'):
                    csv_file_path = os.path.join(depot_note_folder, file)
                    with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                        csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                        csv_reader['Note/10'] = csv_reader['Note/10'].replace('', np.nan)
                        csv_reader['Note/10'] = csv_reader['Note/10'].astype(float)
                        for ligne in csv_reader:
                            ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))                    
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()

                elif file.endswith('.json'):
                    json_file_path = os.path.join(depot_note_folder, file)
                    with open(json_file_path, 'r', encoding='utf-8-sig') as jsonfile:
                        data = json.load(jsonfile)
                        df = pd.DataFrame(data)
                        df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                        for index, row in df.iterrows():
                            ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;",(ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()

                elif file.endswith('.xlsx'):
                    xlsx_file_path = os.path.join(depot_note_folder, file)
                    df = pd.read_excel(xlsx_file_path)
                    df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                    for index, row in df.iterrows():
                        ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()
                else:
                    print(f"Format de fichier non pris en charge: {file}")
                
            else :
                extra_time = False
                if file.endswith('.csv'):
                    csv_file_path = os.path.join(depot_note_folder, file)
                    with open(csv_file_path, 'r', encoding='utf-8-sig') as csvfile:
                        csv_reader = pd.read_csv(csv_file_path, encoding='utf-8-sig')
                        csv_reader['Note/10'] = csv_reader['Note/10'].replace('', np.nan)
                        csv_reader['Note/10'] = csv_reader['Note/10'].astype(float)
                        for ligne in csv_reader:
                            ligne_filtered = {key: value for key, value in ligne.items() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))                    
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()
                            
                elif file.endswith('.json'):
                    json_file_path = os.path.join(depot_note_folder, file)
                    with open(json_file_path, 'r', encoding='utf-8-sig') as jsonfile:
                        data = json.load(jsonfile)
                        df = pd.DataFrame(data)
                        df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                        for index, row in df.iterrows():
                            ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                            cursor_destination = conn.cursor()
                            cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;",(ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            count = cursor_destination.fetchone()[0]
                            if count == 0:
                                print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                            else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                            conn.commit()

                elif file.endswith('.xlsx'):
                    xlsx_file_path = os.path.join(depot_note_folder, file)
                    df = pd.read_excel(xlsx_file_path)
                    df['Note/10'] = df['Note/10'].replace('', np.nan).astype(float)
                    for index, row in df.iterrows():
                        ligne_filtered = {db_column_mapping[key]: row[key] for key in row.keys() if key in db_column_mapping.keys()}
                        cursor_destination = conn.cursor()
                        cursor_destination.execute("SELECT COUNT(*) FROM General WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ?;", (ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        count = cursor_destination.fetchone()[0]
                        if count == 0:
                            print(f"{ligne_filtered['NAME']} {ligne_filtered['SURNAME']} doesn't exist")
                        else:
                                cursor_destination.execute("UPDATE General SET GRADE_LV1=?, LV1=?, EXTRA_TIME=? WHERE (NAME = ? AND SURNAME = ?) OR EMAIL = ? ;", (ligne_filtered["GRADE_LV1"], os.path.splitext(os.path.basename(file))[0], extra_time, ligne_filtered["NAME"], ligne_filtered["SURNAME"], ligne_filtered["EMAIL"]))
                        conn.commit()
                else:
                    print(f"Format de fichier non pris en charge: {file}")

    conn.close()
    return




depot_note_folder ='./data/depot_notes'
db_path ='./data/SQL/data.sqlite3' 

load_survey(depot_note_folder,db_path)

### recuperer list classe & langue

In [42]:
import sqlite3

Data_brute = 'data/SQL/data.sqlite3'
filename = 'CLASS'

conn = sqlite3.connect(Data_brute)
cursor = conn.cursor()
cursor.execute("SELECT DISTINCT(CLASS) FROM General;")
list_Class = cursor.fetchall()
list_Class = [row[0] for row in list_Class]
print(list_Class)
cursor.execute("SELECT DISTINCT(LV2) FROM General;")
list_lv2 = cursor.fetchall()
list_lv2 = [row[0] for row in list_lv2]
print(list_lv2)
conn.close()

['1A', '2A', '1AFT']
['ESPAGNOL', 'ALLEMAND', 'CHINOIS -débutant (jamais étudié)', 'ESPAGNOL -débutant (jamais étudié)']
